In [3]:
# Course : CS 513 - Knowledge Discovery and Data Mining
# Group Members: Branden Bulatao, Joseph Faustino, Natalie Fortes, Isabel Sutedjo
# Id : 20005971, 20006114, 20006007, 20006618
# Purpose : Music Happiness Predictor - Predicts the happiness of music tracks based on various features.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer


# Load dataset
df = pd.read_csv('./dataset.csv')

# Drop unnamed and nulls
df.drop(df.columns[df.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)
df.dropna(inplace=True)

# Convert valence to binary: 0 = sad, 1 = happy
df['valence'] = df['valence'].map(lambda x: 0 if x < 0.5 else 1)

# Convert 'explicit' to int
df['explicit'] = df['explicit'].astype(int)

# One-hot encode categorical features
df = pd.get_dummies(df, columns=['key', 'time_signature', 'track_genre'], drop_first=True)

# Define target and drop non-feature columns
y = df['valence']
X = df.drop(columns=['track_id', 'artists', 'album_name', 'track_name', 'valence'])

# Identify numeric features to scale
numeric_features = ['popularity', 'duration_ms', 'danceability', 'energy', 'loudness',
                    'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo']

# All other features (binary or one-hot) are left as-is
non_scaled_features = [col for col in X.columns if col not in numeric_features]

# ColumnTransformer for selective scaling
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('pass', 'passthrough', non_scaled_features)
    ]
)

# Fit and transform the data
X_processed = preprocessor.fit_transform(X)

# Reconstruct a DataFrame (optional, for inspection/debugging)
X_processed = pd.DataFrame(X_processed, columns=numeric_features + non_scaled_features)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.3, random_state=42)
X

,popularity,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,track_genre_spanish,track_genre_study,track_genre_swedish,track_genre_synth-pop,track_genre_tango,track_genre_techno,track_genre_trance,track_genre_trip-hop,track_genre_turkish,track_genre_world-music
0,73,230666,0,0.676,0.4610,-6.746,0,0.1430,0.0322,0.000001,...,False,False,False,False,False,False,False,False,False,False
1,55,149610,0,0.420,0.1660,-17.235,1,0.0763,0.9240,0.000006,...,False,False,False,False,False,False,False,False,False,False
2,57,210826,0,0.438,0.3590,-9.734,1,0.0557,0.2100,0.000000,...,False,False,False,False,False,False,False,False,False,False
3,71,201933,0,0.266,0.0596,-18.515,1,0.0363,0.9050,0.000071,...,False,False,False,False,False,False,False,False,False,False
4,82,198853,0,0.618,0.4430,-9.681,1,0.0526,0.4690,0.000000,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113995,21,384999,0,0.172,0.2350,-16.393,1,0.0422,0.6400,0.928000,...,False,False,False,False,False,False,False,False,False,True
113996,22,385000,0,0.174,0.1170,-18.318,0,0.0401,0.9940,0.976000,...,False,False,False,False,False,False,False,False,False,True
113997,22,271466,0,0.629,0.3290,-10.895,0,0.0420,0.8670,0.000000,...,False,False,False,False,False,False,False,False,False,True
113998,41,283893,0,0.587,0.5060,-10.889,1,0.0297,0.3810,0.000000,...,False,False,False,False,False,False,False,False,False,True


In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train the Decision Tree Classifier
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Print confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

Accuracy: 78.50%
Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.80      0.80     18571
           1       0.76      0.77      0.77     15629

    accuracy                           0.79     34200
   macro avg       0.78      0.78      0.78     34200
weighted avg       0.79      0.79      0.79     34200

Confusion Matrix:
[[14866  3705]
 [ 3648 11981]]
